## 문제출제 목적
- 내부요인(상품력, 시즌별 상품변화) + 외부요인(날씨,이슈,시청률) 을 통해 프로그램 매출 실적을 사전에 예측하고 대응

### 데이터 설명
- 판매가 0원(무형, 보험 상품)은 추정 제외
- 매주 토요일 18:00 ~ 18:20은 제외(이미 제외되어 있음)
- 편성시간은 10~60분 내외
- 편성표는 6:20~2:20(익일)으로 구성(심야시간대 제외)
- 취급액 = 판매단가 * 주문량

In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 
import locale                                                           

locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8') 

'ko_KR.UTF-8'

In [153]:
file_root = './2020빅콘테스트 문제데이터(데이터분석분야-챔피언리그)/01_제공데이터/'
performance_data = pd.read_excel(file_root+'2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터.xlsx',header=1)
rating_data = pd.read_excel(file_root+'2020 빅콘테스트 데이터분석분야-챔피언리그_시청률 데이터.xlsx',header=1)

In [154]:
display(performance_data.head())
display(rating_data.head())

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0


,시간대,2019-01-01,2019-01-02,2019-01-03,2019-01-04,2019-01-05,2019-01-06,2019-01-07,2019-01-08,2019-01-09,...,2019-12-23,2019-12-24,2019-12-25,2019-12-26,2019-12-27,2019-12-28,2019-12-29,2019-12-30,2019-12-31,2019-01-01 to 2019-12-31
0,02:00,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.003
1,02:01,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.012,0.003
2,02:02,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.000,0.004
3,02:03,0.0,0.0,0.014,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.017,0.0,0.0,0.0,0.0,0.0,0.000,0.004
4,02:04,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.017,0.0,0.0,0.0,0.0,0.0,0.000,0.004


In [174]:
performance_data['노출(분)'].unique()

array([20.        ,         nan, 30.        , 17.1       , 17.08333333,
       27.08333333, 16.1       , 16.08333333, 26.08333333, 16.33333333,
       16.35      , 26.33333333, 16.05      , 26.03333333, 12.13333333,
       16.01666667, 16.03333333, 15.86666667, 16.51666667,  2.46666667,
       16.2       , 15.85      , 15.36666667, 15.83333333, 16.16666667,
       15.66666667, 25.9       , 26.16666667, 25.21666667, 16.        ,
       14.66666667, 24.66666667, 14.7       ,  2.81666667, 14.35      ,
       22.36666667, 15.        , 25.        , 15.01666667, 15.03333333,
       15.06666667, 12.        , 10.        , 22.        , 13.        ,
       23.        ,  3.        , 13.16666667, 12.66666667, 23.2       ,
       13.33333333, 12.5       , 23.16666667,  3.16666667, 13.5       ,
       15.16666667, 15.2       , 25.66666667, 14.83333333, 30.1       ,
       17.66666667,  5.16666667, 25.2       ,  8.85      , 15.5       ,
       15.08333333, 15.25      , 15.33333333,  5.33333333,  3.28

In [183]:
performance_data[performance_data['노출(분)']==30.1]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매량,요일
20906,2019-07-19 08:00:00,30.1,100788,202339,루이띠에 순금 샤이닝 러브 목걸이 35g+골드바 1g,잡화,3650000,11569000.0,3.169589,금요일
20913,2019-07-19 08:30:00,30.1,100788,202339,루이띠에 순금 샤이닝 러브 목걸이 35g+골드바 1g,잡화,3650000,17328000.0,4.747397,금요일
21270,2019-07-22 13:00:00,30.1,100788,202339,루이띠에 순금 샤이닝 러브 목걸이 35g+골드바 1g,잡화,3650000,17326000.0,4.746849,월요일
21277,2019-07-22 13:30:00,30.1,100788,202339,루이띠에 순금 샤이닝 러브 목걸이 35g+골드바 1g,잡화,3650000,0.0,0.000000,월요일
21550,2019-07-24 20:00:00,30.1,100788,202339,루이띠에 순금 샤이닝 러브 목걸이 35g+골드바 1g,잡화,3650000,0.0,0.000000,수요일
...,...,...,...,...,...,...,...,...,...,...
36773,2019-12-24 20:50:00,30.1,100036,200068,구찌 씨마 베이직 양면 울 머플러,잡화,380000,41516000.0,109.252632,화요일
36813,2019-12-25 10:20:00,30.1,100036,200068,구찌 씨마 베이직 양면 울 머플러,잡화,380000,3234000.0,8.510526,수요일
36820,2019-12-25 10:50:00,30.1,100036,200068,구찌 씨마 베이직 양면 울 머플러,잡화,380000,20370000.0,53.605263,수요일
37057,2019-12-28 10:20:00,30.1,100036,200068,구찌 씨마 베이직 양면 울 머플러,잡화,380000,4313000.0,11.350000,토요일


In [180]:
performance_data[performance_data.방송일시 == '2019-08-17 12:40:00']

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매량,요일
24034,2019-08-17 12:40:00,8.85,100348,201089,벨레즈온 브라캡 심리스 원피스 패키지,속옷,29900,971000.0,32.474916,토요일


In [181]:
performance_data.iloc[24034:24040,:]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매량,요일
24034,2019-08-17 12:40:00,8.85,100348,201089,벨레즈온 브라캡 심리스 원피스 패키지,속옷,29900,971000.0,32.474916,토요일
24035,2019-08-17 13:00:00,30.00,100305,200984,오모떼 쉐이핑 브라팬티 10종(7월),속옷,59900,53793000.0,898.046745,토요일
24036,2019-08-17 13:30:00,30.00,100305,200984,오모떼 쉐이핑 브라팬티 10종(7월),속옷,59900,97254000.0,1623.606010,토요일
24037,2019-08-17 14:00:00,20.00,100475,201446,초특가 파뷔에 팬지 대형 후라이팬 세트,주방,59800,19695000.0,329.347826,토요일
24038,2019-08-17 14:20:00,20.00,100475,201446,초특가 파뷔에 팬지 대형 후라이팬 세트,주방,59800,43251000.0,723.260870,토요일
24039,2019-08-17 14:40:00,20.00,100475,201446,초특가 파뷔에 팬지 대형 후라이팬 세트,주방,59800,73601000.0,1230.785953,토요일


In [155]:
# 판매단가가 0인 제품은 처음부터 제외하고 진행
performance_data = performance_data[performance_data.판매단가 != 0].reset_index(drop=True)

In [156]:
performance_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37372 entries, 0 to 37371
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   방송일시    37372 non-null  datetime64[ns]
 1   노출(분)   20588 non-null  float64       
 2   마더코드    37372 non-null  int64         
 3   상품코드    37372 non-null  int64         
 4   상품명     37372 non-null  object        
 5   상품군     37372 non-null  object        
 6   판매단가    37372 non-null  int64         
 7   취급액     37372 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [157]:
# 노출시간이 결측치인 데이터 확인
performance_data[performance_data['노출(분)'].isnull()]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0
5,2019-01-01 06:40:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,9337000.0
26,2019-01-01 14:00:00,NaN,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,20841000.0
28,2019-01-01 14:30:00,NaN,100377,201226,그렉노먼 여성 구스다운 롱 벤치코트,의류,119000,47294000.0
...,...,...,...,...,...,...,...,...
37366,2019-12-31 23:40:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,2328000.0
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0
37369,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0
37370,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0


- 같은 방송에서 함께 여러개의 상품을 판매하는 경우 첫번쨰 상품을 제외한 나머지 상품은 전부 노출시간이 존재하지 않는다. 
- 이런 데이터들을 같은 시간으로 채워넣을지, 아니면 1/n을 할지 등 처리하는 방법을 생각해 봐야함


In [158]:
# 취급액이 0인 데이터(하나도 팔리지 않은 데이터)확인
display(performance_data[performance_data.취급액 == 0])

# 취급액이 판매단가보다 작은 데이터(하나도 팔리지 않은 데이터이지만 전산상 오류로 값이 들어갔다고 함)
display(performance_data[performance_data.취급액<performance_data.판매단가])

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액


,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액
144,2019-01-02 22:00:00,NaN,100148,200432,무이자 LG 울트라HD TV 55UK6800HNC,가전,1440000,50000.0
147,2019-01-02 22:00:00,NaN,100148,200518,일시불 LG 울트라HD TV 70UK7400KNA,가전,2700000,50000.0
148,2019-01-02 22:00:00,NaN,100148,200451,무이자 LG 울트라HD TV 70UK7400KNA,가전,2990000,50000.0
153,2019-01-02 22:20:00,NaN,100148,200518,일시불 LG 울트라HD TV 70UK7400KNA,가전,2700000,50000.0
154,2019-01-02 22:20:00,NaN,100148,200451,무이자 LG 울트라HD TV 70UK7400KNA,가전,2990000,50000.0
...,...,...,...,...,...,...,...,...
36815,2019-12-25 10:20:00,NaN,100036,200070,구찌 인터로킹 GG 탑핸들 체인 숄더 스몰,잡화,2590000,50000.0
37059,2019-12-28 10:20:00,NaN,100036,200070,구찌 인터로킹 GG 탑핸들 체인 숄더 스몰,잡화,2590000,50000.0
37061,2019-12-28 10:20:00,NaN,100039,200073,버버리 홀스페리 페이톤 크로스백,잡화,880000,50000.0
37111,2019-12-28 21:20:00,NaN,100372,201169,(싱글+싱글)일월 품안애 온수매트,생활용품,198000,50000.0


In [159]:
# 위 데이터의 취급액을 0으로 채워줌
zero_idx = performance_data[performance_data.취급액<performance_data.판매단가].index
performance_data.loc[zero_idx,'취급액'] = 0

# 판매량 생성
performance_data['판매량'] = performance_data.취급액/performance_data.판매단가
performance_data

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매량
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,52.606516
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,109.548872
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,81.754386
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.310777
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.218045
...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,68.628378
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,286.117978
37369,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,621.380952
37370,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,87.120253


In [160]:
performance_data.describe()

,노출(분),마더코드,상품코드,판매단가,취급액,판매량
count,20588.000000,37372.000000,37372.000000,3.737200e+04,3.737200e+04,37372.000000
mean,20.051913,100389.497726,201211.879081,4.680006e+05,2.187039e+07,298.009703
std,3.116326,251.374951,737.824966,7.314484e+05,2.019428e+07,422.228815
min,2.466667,100000.000000,200000.000000,1.280000e+04,0.000000e+00,0.000000
25%,20.000000,100153.000000,200539.000000,5.900000e+04,6.880750e+06,19.148208
50%,20.000000,100346.000000,201160.000000,1.099000e+05,1.612950e+07,110.097171
75%,20.000000,100597.000000,201863.000000,5.090000e+05,3.163125e+07,425.476597
max,40.000000,100849.000000,202513.000000,7.930000e+06,3.220090e+08,8070.401003


In [161]:
print('마더코드 수 :',performance_data.마더코드.nunique())
print('상품코드 수 :',performance_data.상품코드.nunique())
print('상품명 수 :',performance_data.상품명.nunique())
print('상품군 수 :',performance_data.상품군.nunique())

마더코드 수 : 687
상품코드 수 : 2042
상품명 수 : 1692
상품군 수 : 11


In [162]:
def date2day(x):
    d = datetime.date(x.year, x.month, x.day) 
    x = d.strftime('%A')
    return x
performance_data['요일'] = performance_data.방송일시.apply(lambda x: date2day(x))
performance_data

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매량,요일
0,2019-01-01 06:00:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,2099000.0,52.606516,화요일
1,2019-01-01 06:00:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,4371000.0,109.548872,화요일
2,2019-01-01 06:20:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,3262000.0,81.754386,화요일
3,2019-01-01 06:20:00,NaN,100346,201079,테이트 여성 셀린니트3종,의류,39900,6955000.0,174.310777,화요일
4,2019-01-01 06:40:00,20.0,100346,201072,테이트 남성 셀린니트3종,의류,39900,6672000.0,167.218045,화요일
...,...,...,...,...,...,...,...,...,...,...
37367,2019-12-31 23:40:00,NaN,100448,201391,일시불쿠첸압력밥솥 6인용,주방,148000,10157000.0,68.628378,화요일
37368,2020-01-01 00:00:00,20.0,100448,201383,무이자쿠첸압력밥솥 10인용,주방,178000,50929000.0,286.117978,수요일
37369,2020-01-01 00:00:00,NaN,100448,201390,일시불쿠첸압력밥솥 10인용,주방,168000,104392000.0,621.380952,수요일
37370,2020-01-01 00:00:00,NaN,100448,201384,무이자쿠첸압력밥솥 6인용,주방,158000,13765000.0,87.120253,수요일


In [163]:
performance_data[(performance_data.요일 == '토요일')]

,방송일시,노출(분),마더코드,상품코드,상품명,상품군,판매단가,취급액,판매량,요일
342,2019-01-05 00:00:00,20.0,100212,200756,스포메틱스 리얼핏 모이스춰라이징 코코넛크림,이미용,49900,10874000.0,217.915832,토요일
343,2019-01-05 00:20:00,20.0,100212,200756,스포메틱스 리얼핏 모이스춰라이징 코코넛크림,이미용,49900,24142000.0,483.807615,토요일
344,2019-01-05 00:40:00,20.0,100212,200756,스포메틱스 리얼핏 모이스춰라이징 코코넛크림,이미용,49900,23462000.0,470.180361,토요일
345,2019-01-05 01:00:00,20.0,100293,200952,일시불[가이거] 제니스시계 주얼리세트,잡화,129000,15067000.0,116.798450,토요일
346,2019-01-05 01:00:00,NaN,100293,200949,무이자[가이거] 제니스시계 주얼리세트,잡화,139000,6857000.0,49.330935,토요일
...,...,...,...,...,...,...,...,...,...,...
37125,2019-12-28 22:00:00,NaN,100372,201184,(퀸사이즈)일월 품안애 온수매트,생활용품,148000,6399000.0,43.236486,토요일
37126,2019-12-28 22:00:00,NaN,100372,201172,(싱글사이즈)일월 품안애 온수매트,생활용품,118000,5059000.0,42.872881,토요일
37127,2019-12-28 22:20:00,20.0,100317,201004,국내산 손질 햇 갑오징어 9팩,농수축,49900,52262000.0,1047.334669,토요일
37128,2019-12-28 22:40:00,20.0,100317,201004,국내산 손질 햇 갑오징어 9팩,농수축,49900,102086000.0,2045.811623,토요일


In [164]:
# 토요일 18:00 방송 데이터는 이미 제거되어있음
idx = []
for i in range(len(performance_data)):
    if (performance_data.요일[i] == '토요일') and (datetime.time(performance_data.방송일시[i].hour,performance_data.방송일시[i].minute) == datetime.time(18,0)):
        idx.append(i)
idx

[]

In [165]:
# 요일별로 팔리는 상품의 취급액 합
d1 = performance_data.groupby('요일')['취급액'].agg([('일별총취급액','sum')]).reset_index()
# 요일별로 어느 가격대의 제품이 팔리는지 파악
d2 = performance_data.groupby('요일')['판매단가'].agg([('일별평균판매단가','mean')]).reset_index()
# 요일별로 어떤 상품군이 
d3 = pd.pivot_table(performance_data, index='요일', columns='상품군', values='판매단가', 
                   aggfunc=np.size, fill_value=0).reset_index()

In [169]:
d1

,요일,일별총취급액
0,금요일,1.083918e+11
1,목요일,1.117374e+11
2,수요일,1.132440e+11
3,월요일,1.105716e+11
4,일요일,1.347699e+11
5,토요일,1.248842e+11
6,화요일,1.137414e+11


In [166]:
# 주말에 가장 비싼 물건이 팔리는 것을 알 수 있다.
d2

,요일,일별평균판매단가
0,금요일,460980.151573
1,목요일,439432.981928
2,수요일,462742.261029
3,월요일,401515.180447
4,일요일,558399.640805
5,토요일,537037.987264
6,화요일,413404.906844


In [167]:
d3

상품군,요일,가구,가전,건강기능,농수축,생활용품,속옷,의류,이미용,잡화,주방,침구
0,금요일,405,817,143,533,463,567,495,179,623,927,60
1,목요일,339,949,168,605,457,514,530,173,573,899,105
2,수요일,420,846,128,578,404,514,547,186,520,1117,180
3,월요일,435,701,97,619,445,594,574,179,515,970,108
4,일요일,382,1261,72,436,405,585,814,209,422,874,108
5,토요일,273,1152,66,503,343,488,813,173,660,696,15
6,화요일,360,753,116,610,360,657,567,206,578,1121,93


In [168]:
count_sum = []
for i in d3.iloc[:,1:].iterrows():
    count_sum.append(sum(i[1]))
    
d3['합계'] = count_sum

for i in range(len(d3)):
    d3.iloc[i,1:-1] = d3.iloc[i,1:]/d3.합계[i]
d3

상품군,요일,가구,가전,건강기능,농수축,생활용품,속옷,의류,이미용,잡화,주방,침구,합계
0,금요일,0.077705,0.156754,0.027437,0.102264,0.088833,0.108787,0.094973,0.034344,0.119532,0.177859,0.011512,5212
1,목요일,0.063818,0.178652,0.031627,0.113893,0.086032,0.096762,0.099774,0.032568,0.107869,0.169239,0.019767,5312
2,수요일,0.077206,0.155515,0.023529,0.106250,0.074265,0.094485,0.100551,0.034191,0.095588,0.205331,0.033088,5440
3,월요일,0.083063,0.133855,0.018522,0.118197,0.084972,0.113424,0.109605,0.034180,0.098339,0.185221,0.020622,5237
4,일요일,0.068606,0.226473,0.012931,0.078305,0.072737,0.105065,0.146193,0.037536,0.075790,0.156968,0.019397,5568
5,토요일,0.052682,0.222308,0.012736,0.097067,0.066191,0.094172,0.156889,0.033385,0.127364,0.134311,0.002895,5182
6,화요일,0.066408,0.138904,0.021398,0.112525,0.066408,0.121195,0.104593,0.038000,0.106622,0.206788,0.017156,5421


In [ ]:
sum(d3)